# Write the results of the Millard DB
***

In [ ]:
import os
from tqdm import tqdm
import pandas as pd
#import pprint
import json 
#pp = pprint.PrettyPrinter(width = 150)
# Generate a list of path of filtered MSA 
path_proteins = "/home/conchae/ML_depolymerase/get_candidates/millard"
path_out = "/home/conchae/ML_depolymerase/get_candidates/millard/DBsuite_depo3"
path_tab = [f"{path_out}/{file}" for file in os.listdir(f"{path_out}") if file[-3:]=="tab"]

out_names = ["query","target","x1","hmm","mismatch","gapopen","query_start","query_stop","target_start","target_stop","evalue","score"]

def scan_tab(query_list) :
    scores = []
    dico_ipr = {}
    depolymerase_dico = {}
    for query in query_list:
        out_names = ["query","target","x1","hmm","mismatch","gapopen","query_start","query_stop","target_start","target_stop","evalue","score"]
        df_tab = pd.read_csv(f"{query}", sep="\t", names= out_names)
        dico_tab = df_tab.to_dict("records")
        for row in dico_tab :
            align_cols = int(row["query_stop"]) - int(row["query_start"])
            if align_cols > 30 and row["score"] > 20 :
                ipr = row["target"].split(".ready")[0]
                prot = row["query"]
                scores.append(row["score"])
                if prot not in depolymerase_dico :
                    positive_out = []
                    positive_out.append(f"{ipr}__{row['score']}")
                    depolymerase_dico[prot] = positive_out
                else :
                    depolymerase_dico[prot].append(f"{ipr}__{row['score']}")
                if ipr not in dico_ipr :
                    sco = []
                    sco.append(row["score"])
                    dico_ipr[ipr]=sco
                else :
                    dico_ipr[ipr].append(row["score"])
    dico_count = {ipr : len(dico_ipr[ipr]) for ipr in dico_ipr}
    return scores, dico_count, dico_ipr, depolymerase_dico

scores_out , dico_count_out, dico_ipr_out, depolymerase_dico_out = scan_tab(path_tab)

with open(f"{path_proteins}/dico_ipr_out.v3.json", "w") as outfile:
    json.dump(dico_ipr_out, outfile)
with open(f"{path_proteins}/dico_ipr_count.v3.json", "w") as outfile:
    json.dump(dico_count_out, outfile)    
with open(f"{path_proteins}/dbsuite_results.v3.json", "w") as outfile:
    json.dump(depolymerase_dico_out, outfile) 
    

    

In [ ]:
import os
import time
import json 
from Bio import SeqIO
from tqdm import tqdm

#pp = pprint.PrettyPrinter(width = 150)
# Generate a list of path of filtered MSA 
path_proteins = "/home/conchae/ML_depolymerase/get_candidates/millard"
path_mill_db = "/home/conchae/databases/Millard_jan_2023/5Jan2023_vConTACT2_proteins.faa"

dico_res = json.load(open(f"{path_proteins}/dbsuite_results.v3.json"))
dico_count = json.load(open(f"{path_proteins}/dico_ipr_count.v3.json"))

proteins_annot = {record.id : " ".join(record.description.split(" ")[1:]) for record in SeqIO.parse(path_mill_db, "fasta") if record.id in dico_res}
proteins_seq = {record.id : record.seq for record in SeqIO.parse(path_mill_db, "fasta") if record.id in dico_res}

# Writting the fasta files :
with open(f"{path_proteins}/millard_depo.v3.fasta", "w") as outfile :
    for prot in tqdm(proteins_seq) :
        outfile.write(f">{prot} {proteins_annot[prot]}\n{proteins_seq[prot]}\n")

# Writting the results files :
#for file in os.listdir(f"{path_proteins}/DBsuite_depo") :
#    if file.split(".")[0] in proteins_annot :
#        os.system(f"cp {path_proteins}/DBsuite_depo/{file} {path_proteins}/millard_dbsuite_results/{file}")
            
dico_annot = {}
for prot in dico_res :
    ipr = [ipr.split("__")[0] for ipr in dico_res[prot]]
    for index, ipr_entry in enumerate(ipr) :
        if ipr_entry not in dico_annot :
            ipr_annot = {proteins_annot[prot] : 1}
            dico_annot[ipr_entry] = ipr_annot
        elif proteins_annot[prot] not in dico_annot[ipr_entry]:
            a = {proteins_annot[prot] : 1}
            dico_annot[ipr_entry].update(a)
        else :
            dico_annot[ipr_entry][proteins_annot[prot]] += 1

for ipr in dico_annot :
    a = {"total_count" : dico_count[ipr]}
    dico_annot[ipr].update(a)

with open(f"{path_proteins}/results_annotation.v3.json", "w") as outfile:
    json.dump(dico_annot, outfile) 
with open(f"{path_proteins}/proteinID_annotation.v3.json", "w") as outfile:
    json.dump(proteins_annot, outfile) 

In [ ]:
#!/bin/bash
#BATCH --job-name=DBsuiste3__
#SBATCH --qos=short
#SBATCH --ntasks=1 
#SBATCH --cpus-per-task=5 
#SBATCH --mem=10gb 
#SBATCH --time=01-00:00:00 
#SBATCH --output=DBsuiste3__%j.log 

source /storage/apps/ANACONDA/anaconda3/etc/profile.d/conda.sh
conda activate bio_phylo

python /home/conchae/ML_depolymerase/scripts/results_v3.py

In [ ]:
# Build the sequence idex file :
import os 
from Bio import SeqIO
import pandas as pd 

path_millard = "/home/conchae/ML_depolymerase/get_candidates/millard"

data = {}
for rec in SeqIO.parse(f"{path_millard}/millard_depo.v3.fasta" , "fasta") :
    if rec.seq not in data :
        data[rec.seq] = [rec.id.split("__")[0]]
    else :
        data[rec.seq].append(rec.id.split("__")[0])
        
with open(f"{path_millard}/df_sequences.index.v3.csv" ,"w") as outfile :
    for index_seq, seq in enumerate(list(data.keys())) :
        for prot in data[seq] :
            outfile.write(f"{index_seq}\t{prot}\t{seq}\n")
        
df = pd.read_csv(f"{path_millard}/df_sequences.index.v3.csv", sep="\t", names = ["index","id","sequence"])       
df = df.drop_duplicates(subset=["index"], keep="first")
df.to_csv(f"{path_millard}/df_sequences.index.clean.v3.csv", sep="\t", columns = ["index","sequence"], index=False)


df_2 = pd.read_csv(f"{path_millard}/df_sequences.index.clean.v2.csv", sep="\t")
with open(f"{path_millard}/millard_depo.indexed.v3.fasta" , "w") as outfile :
    dico_interest = df.to_dict("records")
    for row in dico_interest :
        outfile.write(f">{row['index']}\n{row['sequence']}\n")

In [ ]:
import os
import subprocess
from tqdm import tqdm
from random import sample
import time
import pandas as pd
from multiprocessing.pool import ThreadPool

path_trunc = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/Resdom/truncated_msa"
path_decipher = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"
path_db = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/15122022_session"

results_df = pd.read_csv(f"{path_db}/Results_III_DataFrame.csv", sep="\t", header = 0)
sequence_df = pd.read_csv(f"{path_db}/Results_III_sequences.csv", sep = "\t" ,names= ["index", "sequence"])


def f_path_tab(query) :
    "From the protein name, get the tab output with score >= 20"
    path_decipher = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"
    strain , prophage = query.split("__")[0], query.split("__")[1]
    path_tab = f"{path_decipher}/{strain}/DBsuite_depo_v3/{prophage}/{query}.suite.tab"
    out_names = ["query","target","x1","hmm","mismatch","gapopen","consensus_start","consensus_stop","target_start","target_stop","evalue","score"]
    df_tab = pd.read_csv(f"{path_tab}", sep="\t", names= out_names)
    df_tab = df_tab[df_tab["score"] >= 20]
    sequence = open(f"{path_decipher}/{strain}/tmp/{prophage}/{query}.fasta").read().split("\n")[1]
    return df_tab, sequence

def get_qlimits(query,target) :
    strain , prophage = query.split("__")[0], query.split("__")[1]
    df_tab, sequence = f_path_tab(query)
    #print(df_tab, target)
    #return (1,2)
    consensus_start = df_tab[df_tab["target"]==f"{target}"]["consensus_start"].values[0] 
    consensus_stop = df_tab[df_tab["target"]==f"{target}"]["consensus_stop"].values[0]
    msa_seq = "".join(open(f"{path_decipher}/{strain}/mmseqs_out/{prophage}/{query}.MSA.a2m").read().split(">")[1].split("\n")[1:])
    i = 0
    for n in range(len(msa_seq)) :
        if msa_seq[n] == "-" :
            pass
        else :
            i += 1 
        if n == int(consensus_start)-1 :
            qstart = i
        elif n == int(consensus_stop)-1 :
            qstop = i 
            break            
    return qstart, qstop

In [ ]:
import os 
import pandas as pd 
import json

path_labels = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_labeling/phageboost"
path_db = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/15122022_session"
path_decipher = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"

f_labels=["Prophage_name","KL_ancestor","Id_monophyletic_group","Number of clades","Number of leafs","Number of new ancestors","Number of k-type swap","Nodes k-types","Nodes k-types all"]
df_prophages = pd.read_csv(f"{path_labels}/prophage_data.clusters_80.phageboost_70.tsv", sep="\t", names =f_labels) 

results_DBsuite = json.load(open(f"{path_db}/dbsuite_results.v3.json"))

def f_path_tab(query) :
    "From the protein name, get the tab output with score >= 20"
    path_decipher = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"
    strain , prophage = query.split("__")[0], query.split("__")[1]
    path_tab = f"{path_decipher}/{strain}/DBsuite_depo_v3/{prophage}/{query}.suite.tab"
    out_names = ["query","target","x1","hmm","mismatch","gapopen","consensus_start","consensus_stop","target_start","target_stop","evalue","score"]
    df_tab = pd.read_csv(f"{path_tab}", sep="\t", names= out_names)
    df_tab = df_tab[df_tab["score"] >= 20]
    sequence = open(f"{path_decipher}/{strain}/tmp/{prophage}/{query}.fasta").read().split("\n")[1]
    return df_tab, sequence

def get_qlimits(query,target) :
    strain , prophage = query.split("__")[0], query.split("__")[1]
    df_tab, sequence = f_path_tab(query)
    #print(df_tab, target)
    #return (1,2)
    consensus_start = df_tab[df_tab["target"]==f"{target}"]["consensus_start"].values[0] 
    consensus_stop = df_tab[df_tab["target"]==f"{target}"]["consensus_stop"].values[0]
    msa_seq = "".join(open(f"{path_decipher}/{strain}/mmseqs_out/{prophage}/{query}.MSA.a2m").read().split(">")[1].split("\n")[1:])
    i = 0
    for n in range(len(msa_seq)) :
        if msa_seq[n] == "-" :
            pass
        else :
            i += 1 
        if n == int(consensus_start)-1 :
            qstart = i
        elif n == int(consensus_stop)-1 :
            qstop = i 
            break            
    return qstart, qstop
        
# df 1 : [protein name, #sequence_index# , KL type ancestor, ancestor_id, IPR, score, qstart, qstop, sequence_aa]
with open(f"{path_db}/Results_III_DataFrame.v3.csv", "w") as outfile :
    for protein in tqdm(results_DBsuite) :
        df_tab , sequence = f_path_tab(protein)
        prophage = "__".join(protein.split("__")[0:2])+".fasta"
        df_prot = df_prophages[df_prophages["Prophage_name"]==prophage]
        kl_ancestor = df_prot["KL_ancestor"].values[0]
        ancestor_id = df_prot["Id_monophyletic_group"].values[0]
        #print(prophage, kl_ancestor , ancestor_id)
        for index_tab, hit_ipr in df_tab.iterrows() :
            target = hit_ipr["target"]
            start , stop = get_qlimits(protein,target)
            score = hit_ipr["score"]
            if stop - start >= 30 :
                outfile.write(f"{protein}\t{kl_ancestor}\t{ancestor_id}\t{target}\t{score}\t{start}\t{stop}\t{sequence}\n")
                #print(f"{protein}\t{kl_ancestor}\t{ancestor_id}\t{target}\t{score}\t{start}\t{stop}\n")         
                
                
labels_results = ["protein_name","KL_type_ancestor","ancestor_id","IPR_entry","score","qstart","qstop","sequence"]
results_df = pd.read_csv(f"{path_db}/Results_III_DataFrame.v3.csv", sep="\t", names= labels_results)

sequences_df = results_df["sequence"].unique()
with open(f"{path_db}/Results_III_sequences.v3.csv", "w") as outfile :
    for index, sequence in enumerate(sequences_df) :
        print(sequence)
        outfile.write(f"{index}\t{sequence}\n")

# *******************
# DF with index sequences :

import pandas as pd 
import os 
path_labels = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_labeling/phageboost"
path_db = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/15122022_session"
path_decipher = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"

labels_results = ["protein_name","KL_type_ancestor","ancestor_id","IPR_entry","score","qstart","qstop","sequence"]
results_df = pd.read_csv(f"{path_db}/Results_III_DataFrame.csv", sep="\t", names= labels_results)
sequence_df = pd.read_csv(f"{path_db}/Results_III_sequences.csv", sep = "\t" ,names= ["index", "sequence"])

results_df["index_seq"] = sequence_df[sequence_df["sequence"]== results_df["sequence"]]["index"]

index_list = []
for index , row in tqdm(results_df.iterrows()) :
    index_list.append(sequence_df[sequence_df["sequence"]== row["sequence"]]["index"].values[0])
    
results_df["index_seq"] = index_list
results_df.to_csv(f"{path_db}/Results_III_DataFrame.csv", sep="\t",header =  ["protein_name","KL_type_ancestor","ancestor_id","IPR_entry","score","qstart","qstop","sequence","index_seq"], index = False )

